In [ ]:
from decimal import Decimal, ROUND_HALF_UP

import cv2
import numpy as np
from scipy.spatial import Delaunay
from scipy.stats import norm
from tqdm import tqdm


class DT_PIV(object):

    def __init__(self, img_path, out_path, filename, fmt, n):
        self.img_path = img_path
        self.out_path = out_path
        self.filename = filename
        self.fmt = fmt
        self.n = n

    def run(self):
        print("\n\n")

        for i in tqdm(range(1, self.n, 1)):  # 0 ~ n-1 -> range(0, self.n)
            itr = 1

            ############
            # read img #
            ############
            img_a = cv2.imread(f"{self.img_path}{self.filename}{i:06}.{self.fmt}", 0)  # 00000i -> {i:06}
            img_b = cv2.imread(f"{self.img_path}{self.filename}{i + itr:06}.{self.fmt}", 0)

            img_a = img_a[100:, 140:900]
            img_b = img_b[100:, 140:900]

            #######
            # pmc #
            #######
            particle_position_a = self.pmc(img=img_a, threshold=0.75)
            particle_position_b = self.pmc(img=img_b, threshold=0.75)

            #########################
            # Delaunay Tessellation #
            #########################
            tri_info_a, tri_center_a = self.delaunay_tessellation(pts=particle_position_a)
            tri_info_b, tri_center_b = self.delaunay_tessellation(pts=particle_position_b)

            ###########################
            # piv (cross-correlation) #
            ###########################
            G_X, G_Y, G_dX, G_dY, flag = self.piv(
                img_a=img_a_gauss,
                img_b=img_b_gauss,
                grid_nums=(25, 48),
                win_sizes=(16, 16),
                search_win_sizes=(8, 8),
                threshold=0.5,
            )

            # np.savetxt(f"{self.out_path}{self.filename}{i}_std_piv_dx.csv", G_dX, delimiter=',')
            # np.savetxt(f"{self.out_path}{self.filename}{i}_std_piv_dy.csv", G_dY, delimiter=',')

            ###################
            # post-processing #
            ###################
            G_dX, G_dY = self.post(dX=G_dX, dY=G_dY, flag=flag)

            np.savetxt(f"{self.out_path}{self.filename}{i}_std_piv_dx_2.csv", G_dX, delimiter=',')
            np.savetxt(f"{self.out_path}{self.filename}{i}_std_piv_dy_2.csv", G_dY, delimiter=',')

            #################
            # interpolation #
            #################
            P_dXdY = self.interpolate_vectors(G_X=G_X, G_Y=G_Y, G_dX=G_dX, G_dY=G_dY, pp=particle_position_a)

            # img_a_b = np.zeros_like(img_a)
            # pp = particle_position_a + P_dXdY
            # for s, t in zip(pp[:, 0].astype(int), pp[:, 1].astype(int)):
            #     if 0 <= s < 256 and 0 <= t < 256:
            #         img_a_b[t, s] = 255

            # cv2.imwrite(f"{self.out_path}{self.filename}{i + itr}_pred.bmp", img_a_b)

            ###########################
            # nearest-neighbor method #
            ###########################
            result = self.nn_method(pp_a=particle_position_a, pp_b=particle_position_b, P_dXdY=P_dXdY, s=3)

            ###############
            # save result #
            ###############
            np.savetxt(f"{self.out_path}{self.filename}{i:06}.csv", result, delimiter=',', header="x, y, dX, dY")

    @staticmethod
    def delaunay_tessellation(pts):
        tri = Delaunay(pts)
        tri_info = pts[tri.simplices]
        tri_center = tri_info.mean(axis=(1, 2))

        return tri_info, tri_center

    @staticmethod
    def piv(tri_info_a, tri_info_b, grid_nums, win_sizes, search_win_sizes, threshold=0.7, r_mode=False, dx=None, dy=None):
        pass

    @staticmethod
    def post(dX, dY, flag):
        pass

    @staticmethod
    def pmc(img, threshold=0.7):
        def gauss_circle(image, sd, high=255, low=0, random_sd=0):
            height, width = image.shape[:2]
            rd = np.random.normal(0, random_sd, (height, width))
            scale = high - low
            s = 1 / norm.pdf(0, loc=0, scale=sd)

            for y in range(0, height):
                for x in range(0, width):
                    dist = np.sqrt((x - width // 2) ** 2 + (y - height // 2) ** 2)
                    n = norm.pdf(dist, loc=0, scale=sd)
                    image[y, x] = int(np.clip(n * s * scale + low + rd[y, x], 0, 255))
            return image

        def sub_pixel_analysis(ji, cc, t_size):
            res = np.zeros((1, 2))

            for j, i in zip(ji[0], ji[1]):
                if 0 < j < cc.shape[0] - 1 and 0 < i < cc.shape[1] - 1:
                    # judge whether peak value or not among 3x3
                    if cc[j, i] == np.amax(cc[j - 1:j + 2, i - 1:i + 2]):
                        eps_x = 0.5 * (np.log(cc[j, i - 1]) - np.log(cc[j, i + 1])) \
                                / (np.log(cc[j, i - 1]) + np.log(cc[j, i + 1]) - 2 * np.log(cc[j, i]))
                        eps_y = 0.5 * (np.log(cc[j - 1, i]) - np.log(cc[j + 1, i])) \
                                / (np.log(cc[j - 1, i]) + np.log(cc[j + 1, i]) - 2 * np.log(cc[j, i]))

                        # round half-up
                        eps_x = float(Decimal(str(eps_x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP))
                        eps_y = float(Decimal(str(eps_y)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP))

                        res = np.vstack([res, [i + eps_x + t_size[1] // 2, j + eps_y + t_size[0] // 2]])

            return np.unique(res[1:, :], axis=0)  # Exclude duplicates just in case...

        tracer_img = np.zeros((15, 15), dtype=np.uint8)
        tracer_img = gauss_circle(image=tracer_img, sd=2)

        c = cv2.matchTemplate(img, tracer_img, cv2.TM_CCORR_NORMED)
        c_j, c_i = np.where(c > threshold)

        p_xy = sub_pixel_analysis(ji=[c_j, c_i], cc=c, t_size=tracer_img.shape)

        return p_xy
